Kaggle Boilerplate... (I left it as-is)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Install requirements to get Object Detection example running -- found out after errors that I also needed to separately install icevision and effdet. (Also mentioned in https://github.com/PyTorchLightning/lightning-flash/issues/803 and https://github.com/PyTorchLightning/lightning-flash/issues/949)

In [ ]:
pip install 'git+https://github.com/PyTorchLightning/lightning-flash.git#egg=lightning-flash[image]' icevision effdet

Ok, now I am starting following the code at https://lightning-flash.readthedocs.io/en/latest/reference/object_detection.html

In [1]:
import flash
from flash.core.data.utils import download_data
from flash.image import ObjectDetectionData, ObjectDetector

In [2]:

datamodule = ObjectDetectionData.from_coco(
    train_folder="../input/plantdoc/train/",
    train_ann_file="../input/plantdoc/train/_annotations.coco.json",
    val_split=0.1,
    transform_kwargs={"image_size": 512},
    batch_size=64,
)

In [3]:
import torch
# 2. Build the task
model = ObjectDetector(
    head='yolov5',
    backbone='small',
    optimizer=torch.optim.AdamW,
    learning_rate=5e-5,
    num_classes=datamodule.num_classes,
    image_size=512,
)

Trained, and ran fine.

In [ ]:
import pytorch_lightning as pl
print(pl.__version__)
logger = pl.loggers.CSVLogger(save_dir='logs/')
                              
# 3. Create the trainer and finetune the model
trainer = flash.Trainer(
    max_epochs=100,
    gpus=1,
    precision=16,
    accumulate_grad_batches=24,
    logger=logger,
    val_check_interval=0.5,
)
trainer.finetune(model, datamodule=datamodule)
trainer.save_checkpoint("object_detection.pt")

In [16]:
import pandas as pd
metrics = pd.read_csv(f'./logs/default/version_0/metrics.csv')
display(metrics.head())

aggreg_metrics = []
agg_col = "epoch"
for i, dfg in metrics.groupby(agg_col):
    agg = dict(dfg.mean())
    agg[agg_col] = i
    aggreg_metrics.append(agg)

df_metrics = pd.DataFrame(aggreg_metrics)
df_metrics[['val_val_loss', 'train_train_loss']].plot(grid=True, legend=True, xlabel=agg_col)
df_metrics[['val_precision (IoU=0.50:0.95,area=all)', 'val_recall (IoU=0.50:0.95,area=all,maxDets=100)']].plot(grid=True, legend=True, xlabel=agg_col)

In [17]:
datamodule = ObjectDetectionData.from_files(
    predict_files=[
        "../input/plant-pathology-2021-fgvc8-960px/test_images/85f8cb619c66b863.jpg",
        "../input/plant-pathology-2021-fgvc8-960px/test_images/ad8770db05586b59.jpg",
        "../input/plant-pathology-2021-fgvc8-960px/test_images/c7b03e718489f3ca.jpg",
    ],
    transform_kwargs={"image_size": 128},
    batch_size=4,
)
predictions = trainer.predict(model, datamodule=datamodule)

First of all, the length of `predictions` is 1. Ok, let's look at the first item...

In [18]:
predictions[0][0]

This looks very similar to COCO format, so maybe it could be improved by following COCO format more precisely --- but as Python objects (I think a dictionary)

But here's the truly unexpected part: The example prepares 3 files, yet 2 predictions are returned. What happened??